In [3]:
import pandas as pd 
import numpy as np
import pyecharts
import plotly.express as px # visualization
import pandas_datareader as pdr # access fred
from datetime import date
from fredapi import Fred
import requests
import openpyxl

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

from pyecharts import options as opts
from pyecharts.charts import Bar, Line

In [4]:
fred_key = 'a4459da91e23d9116f98724098d92428'
fred = Fred(api_key=fred_key)
today = date.today()


# Get Series Data
def get_fred_series_data(fred_key,series, **kwargs):
  # url
  url = "https://api.stlouisfed.org/fred/series/observations?series_id={}&api_key={}&file_type=json".format(series, fred_key)

  # response
  response = requests.get(url)
  data = response.json()
  return data


def get_fred_data(param_list, start_date, end_date):
  df = pdr.DataReader(param_list, 'fred', start_date, end_date)
  return df.reset_index()

In [2]:
quaterly_data = pd.read_excel('C:/Users/fanjz/Desktop/dataset/gdp/quarterly_gdp_data.xlsx')
anuual_data = pd.read_excel('C:/Users/fanjz/Desktop/dataset/gdp/yearly_gdp_data.xlsx')

In [5]:
code_all = pd.read_excel('C:/Users/fanjz/Desktop/dataset/gdp/gdp_code.xlsx', sheet_name='all')

code_annual = code_all[code_all['Frequency'] == 'Annual']
code_quarter = code_all[code_all['Frequency'] == 'Quarterly']

code_annual_list = code_annual['Code'].tolist()
code_quarter_list = code_quarter['Code'].tolist()

In [6]:
data_annual = get_fred_data(code_annual_list, start_date='1950-01-01', end_date=str(today))
data_quarter = get_fred_data(code_quarter_list, start_date='1950-01-01', end_date=str(today))

In [15]:
data_annual =  data_annual.rename(columns={'DATE' : 'Date'})
data_quarter =  data_quarter.rename(columns={'DATE' : 'Date'})

In [16]:
data_annual_df = data_annual.melt(id_vars='Date', var_name='Code', value_name='Values')
data_quarter_df = data_quarter.melt(id_vars='Date', var_name='Code', value_name='Values')

In [17]:
data_annual_merge = pd.merge(data_annual_df, code_annual, on='Code')
data_quarter_merge = pd.merge(data_quarter_df, code_quarter, on='Code')

In [18]:
data_quarter_merge['Year'] = data_quarter_merge['Date'].dt.year
data_quarter_merge['Quarter'] = pd.PeriodIndex(data_quarter_merge['Date'], freq='Q').quarter
data_quarter_merge['Year_quarter'] = data_quarter_merge['Year'].astype(str) + 'Q' + data_quarter_merge['Quarter'].astype(str)

In [19]:
data_annual_merge = data_annual_merge.dropna(subset=['Values'])
data_quarter_merge = data_quarter_merge.dropna(subset= ['Values'])

data_annual_merge['Date'] = data_annual_merge['Date'].dt.strftime('%Y-%m-%d')
data_quarter_merge['Date'] = data_quarter_merge['Date'].dt.strftime('%Y-%m-%d')


In [20]:
data_quarter_merge

,Date,Code,Values,Name,Frequency,Type,Unit,Index Mapping,Index Level,Indicator,Country,URL,Year,Quarter,Year_quarter
0,1950-01-01,GDP,280.828,Gross domestic product,Quarterly,Nominal,Dollar (Billions),NaN,0.0,GDP,US,https://fred.stlouisfed.org/series/GDP,1950,1,1950Q1
1,1950-04-01,GDP,290.383,Gross domestic product,Quarterly,Nominal,Dollar (Billions),NaN,0.0,GDP,US,https://fred.stlouisfed.org/series/GDP,1950,2,1950Q2
2,1950-07-01,GDP,308.153,Gross domestic product,Quarterly,Nominal,Dollar (Billions),NaN,0.0,GDP,US,https://fred.stlouisfed.org/series/GDP,1950,3,1950Q3
3,1950-10-01,GDP,319.945,Gross domestic product,Quarterly,Nominal,Dollar (Billions),NaN,0.0,GDP,US,https://fred.stlouisfed.org/series/GDP,1950,4,1950Q4
4,1951-01-01,GDP,336.000,Gross domestic product,Quarterly,Nominal,Dollar (Billions),NaN,0.0,GDP,US,https://fred.stlouisfed.org/series/GDP,1951,1,1951Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54346,2023-01-01,A799RL1Q225SBEA,11.500,Gross investment,Quarterly,Real,Pct,State and local,3.0,GDP,US,https://fred.stlouisfed.org/series/A799RL1Q225...,2023,1,2023Q1
54347,2023-04-01,A799RL1Q225SBEA,18.700,Gross investment,Quarterly,Real,Pct,State and local,3.0,GDP,US,https://fred.stlouisfed.org/series/A799RL1Q225...,2023,2,2023Q2
54348,2023-07-01,A799RL1Q225SBEA,16.000,Gross investment,Quarterly,Real,Pct,State and local,3.0,GDP,US,https://fred.stlouisfed.org/series/A799RL1Q225...,2023,3,2023Q3
54349,2023-10-01,A799RL1Q225SBEA,22.200,Gross investment,Quarterly,Real,Pct,State and local,3.0,GDP,US,https://fred.stlouisfed.org/series/A799RL1Q225...,2023,4,2023Q4


In [21]:
data_quarter_merge.to_excel('C:/Users/fanjz/Desktop/dataset/gdp/quarterly_gdp_data.xlsx', index=False)
data_annual_merge.to_excel('C:/Users/fanjz/Desktop/dataset/gdp/yearly_gdp_data.xlsx', index=False)